In [1]:
from ctransformers import AutoModelForCausalLM

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = AutoModelForCausalLM.from_pretrained(
    "TheBloke/Mistral-7B-Instruct-v0.1-GGUF", 
    model_file="/run/media/joramas/DATA/THESIS/playground/models/mistral-7b-instruct-v0.1.Q5_K_M.gguf", 
    model_type="mistral", 
    gpu_layers=30)


/home/joramas/miniconda/envs/thesis/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 9532.51it/s]
Fetching 0 files: 0it [00:00, ?it/s]


In [2]:
import os
import re

# Define a function to preprocess text
def preprocess_text(text):
    
    text = text.lower()
    # Split the text into sentences
    sentences = re.split(r'[.!?]', text)
    
    for i,sentence in enumerate(sentences):
        sentence = sentence.split("|")[-1]
        sentence = re.sub(r'[^a-zA-Z\s]', '', sentence)
        sentences[i] = sentence.split("|")[-1]
    
    return sentences

# Load the dataset from a folder
dataset_folder = "pipeline/topic_model/test_dataset"
sentences = []

# Iterate over files in the dataset folder
for filename in os.listdir(dataset_folder):
    if filename.endswith(".txt"):
        with open(os.path.join(dataset_folder, filename), "r", encoding="utf-8") as file:
            text = file.read()
            sentences.extend(preprocess_text(text))



print(f'length of documents: {len(sentences)}')
print(f"first sentences:\n", "\n".join(sentences[:5]))

length of documents: 107599
first sentences:
 breast cancer risk test devised httpbbc
gp workload harming care  bma poll httpbbc
short peoples heart risk greater httpbbc
new approach against hiv promising httpbbc
coalition undermined nhs  doctors httpbbc


In [3]:
from pipeline.embeddings.basic_embeddings import Embedding

embed_model = Embedding()

embeddings = []

for sentence in sentences:
    embeddings.append(embed_model.embed(sentence))

print(embeddings[:5])

[[1, 13886, 8875, 4623, 1369, 1847, 2458, 3550, 1754, 28717], [1, 319, 28720, 771, 1768, 6241, 288, 1656, 28705, 287, 705, 8175, 3550, 1754, 28717], [1, 2485, 22529, 3031, 4623, 6517, 3550, 1754, 28717], [1, 633, 4431, 1835, 295, 449, 22449, 3550, 1754, 28717], [1, 28131, 640, 858, 1311, 307, 6270, 28705, 13500, 3550, 1754, 28717]]


In [4]:
print(embeddings[2])

[1, 2485, 22529, 3031, 4623, 6517, 3550, 1754, 28717]


In [5]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import umap


# embeddings = umap.UMAP(n_neighbors=15, 
#                     n_components=10, 
#                     min_dist=0.0, 
#                     metric='cosine',
#                    ).fit_transform(embeddings)


scores = []
for k in range(5,6):
    kmeans = KMeans(n_clusters=k, random_state=42).fit(embeddings)
    score = silhouette_score(embeddings, kmeans.labels_)
    scores.append(score)

best_k = scores.index(max(scores)) + 5 
documents = KMeans(n_clusters=best_k, random_state=42).fit(embeddings)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (107599,) + inhomogeneous part.

In [ ]:
best_k

5

In [ ]:
documents.__dict__

{'n_clusters': 5,
 'init': 'k-means++',
 'max_iter': 300,
 'tol': 0.0001,
 'n_init': 'warn',
 'verbose': 0,
 'random_state': 42,
 'copy_x': True,
 'algorithm': 'lloyd',
 'n_features_in_': 10,
 '_tol': 0.000643173360824585,
 '_n_init': 10,
 '_algorithm': 'lloyd',
 '_n_threads': 6,
 'cluster_centers_': array([[ 8.648771 ,  2.9892354,  8.056784 ,  1.4812537,  3.6511312,
          3.829946 ,  1.9802415,  8.671658 ,  5.114523 ,  4.1643057],
        [ 9.655565 ,  4.249937 ,  4.2360334,  1.9627676, 18.252346 ,
          5.0754657,  4.3221054,  4.6452446,  5.5084286,  5.4275656],
        [ 7.258113 , 19.762693 ,  6.308504 ,  3.41842  ,  4.9693966,
          5.3060803,  5.550563 ,  5.112668 ,  7.8142514,  4.569195 ],
        [ 9.942116 ,  4.468465 ,  3.7789404,  3.2886782,  4.557868 ,
          5.314192 ,  5.188673 ,  4.111411 ,  4.2608705,  4.9704103],
        [ 9.739618 ,  3.7230263,  4.303022 ,  3.1084948,  2.9522576,
          3.1164143, 17.178328 ,  9.0201435, 11.967562 ,  4.3536453]],
   

In [ ]:
from pipeline.embeddings.basic_embeddings import Embedding

embed_model = Embedding()

# select the topic words and decode them
topic_words = []
for i in range(best_k):
    topic_words.append([embed_model.decode(word) for word in embeddings[documents.labels_ == i].mean(axis=0).argsort()[-10:][::-1]])
print(topic_words)


AttributeError: 'SentenceTransformer' object has no attribute 'decode'

In [ ]:
# k * vocab
X_per_cluster = model.transform(documents)
# D * vocab
X_origin = self.vectorizer_model.transform(origin_documents)

if self.word_select_method == 'tfidf_idfi':
    socres = TFIDF_IDFi(X_per_cluster, X_origin, documents).socre()
elif self.word_select_method == 'tfidf_tfi':
    socres = TFIDF_TFi(X_per_cluster, X_origin, all_documents).socre()
elif self.word_select_method == 'tfi':
    socres = TFi(X_per_cluster).socre()
elif self.word_select_method == 'tfidfi':
    socres = TFIDFi(X_per_cluster).socre()